In [47]:
import tkinter as tk
import numpy as np
import cv2
from PIL import Image, ImageDraw
from keras.models import load_model


In [23]:
# Load the trained CNN model
model = load_model('C:/Users/ashis/mnist.h5')
print("CNN model loaded successfully")

CNN model loaded successfully


In [51]:

# Mouse drawing event
def event_function(event):
    x, y = event.x, event.y
    r = 15
    canvas.create_oval(x - r, y - r, x + r, y + r, fill='white', outline='white')
    img_draw.ellipse((x - r, y - r, x + r, y + r), fill='white')

# Clear canvas and image
def clear():
    global img, img_draw
    canvas.delete('all')
    img = Image.new('L', (500, 500), 0)
    img_draw = ImageDraw.Draw(img)
    label_status.config(text='PREDICTED DIGIT: NONE')

# Predict digit
def predict():
    img_array = np.array(img)
    img_array = cv2.resize(img_array, (28, 28))
    img_array = cv2.GaussianBlur(img_array, (5, 5), 0)
    img_array = img_array.astype('float32') / 255.0
    _, img_array = cv2.threshold(img_array, 0.2, 1.0, cv2.THRESH_BINARY)

    # Centering logic
    def center_image(img):
        coords = cv2.findNonZero((img * 255).astype('uint8'))
        if coords is not None:
            x, y, w, h = cv2.boundingRect(coords)
            cropped = img[y:y+h, x:x+w]
            centered = np.zeros((28, 28), dtype='float32')
            offset_x = (28 - w) // 2
            offset_y = (28 - h) // 2
            centered[offset_y:offset_y + h, offset_x:offset_x + w] = cropped
            return centered
        return img

    img_array = center_image(img_array)
    img_array = img_array.reshape(1, 28, 28, 1)

    result = model.predict(img_array)[0]
    label = int(np.argmax(result))
    confidence = result[label] * 100

    label_status.config(text=f'PREDICTED DIGIT: {label} ({confidence:.2f}%)')



# GUI setup
win = tk.Tk()
win.title("Digit Recognizer - CNN Version")

canvas = tk.Canvas(win, width=500, height=500, bg='black')
canvas.grid(row=0, column=0, columnspan=4)

button_predict = tk.Button(win, text='PREDICT', bg='blue', fg='white', font='Helvetica 20 bold', command=predict)
button_predict.grid(row=1, column=1)

button_clear = tk.Button(win, text='CLEAR', bg='orange', fg='white', font='Helvetica 20 bold', command=clear)
button_clear.grid(row=1, column=2)

label_status = tk.Label(win, text='PREDICTED DIGIT: NONE', bg='white', font='Helvetica 24 bold')
label_status.grid(row=2, column=0, columnspan=4)

canvas.bind('<B1-Motion>', event_function)

# Blank image for drawing
img = Image.new('L', (500, 500), 0)
img_draw = ImageDraw.Draw(img)

win.mainloop()




